In [0]:
%run "../includes/configuration"

#### Obtener el DF principal: Movies

In [0]:
movie_df = spark.read.parquet(f"{silver_foler_path}/movies").filter("year_release_date >= 2000")

#### Obtener los datos de Languages

In [0]:
movie_languages_df = spark.read.parquet(f"{silver_foler_path}/movie_languages")
languages_df = spark.read.parquet(f"{silver_foler_path}/language")

#### Obtener los datos de Genre

In [0]:
genre_df = spark.read.parquet(f"{silver_foler_path}/genre")
movie_genre = spark.read.parquet(f"{silver_foler_path}/movie_genre")

#### Obtener Tablas Cruzadas

In [0]:
languages_final = movie_languages_df.join(languages_df,movie_languages_df.language_id == languages_df.language_id,"inner") \
                                        .select(movie_languages_df.movie_id, movie_languages_df.language_id, languages_df.language_name)

In [0]:
genre_final = movie_genre.join(genre_df,movie_genre.genre_id == genre_df.genre_id,"inner") \
                                .select(movie_genre.movie_id, movie_genre.genre_id, genre_df.genre_name)

#### Obtener tabla final

In [0]:
  movie_languages_final = movie_df.join(languages_final,movie_df.movie_id == languages_final.movie_id,"inner") \
                          .select(movie_df.title, movie_df.duration_time, movie_df.release_date,movie_df.vote_average,
                                  languages_final.language_name,movie_df.movie_id)

In [0]:
from pyspark.sql.functions import current_timestamp

movie_final = movie_languages_final.join(genre_final, movie_languages_final.movie_id == genre_final.movie_id,"inner") \
                                  .select(movie_languages_final.title, movie_languages_final.duration_time, movie_languages_final.release_date,
                                          movie_languages_final.vote_average, movie_languages_final.language_name, genre_final.genre_name).withColumn("created_date",current_timestamp()).orderBy(movie_languages_final.release_date, ascending=False)

In [0]:
display(movie_final)

In [0]:
movie_final.write.mode("overwrite").parquet(f"{gold_folder_path}/result_movie_genre_language")

In [0]:
display(spark.read.parquet(f"{gold_folder_path}/result_movie_genre_language"))